In [1]:
import xml.etree.cElementTree as et
import pandas as pd
import numpy as np
import requests

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics import tsaplots

from scipy.stats import norm

from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.debugger import set_trace
#from statsmodels.stats.proportion import proportion_ztest

# Step 1: Acquire Data From CSV File

In [2]:
beach_complete = pd.read_csv('beach_complete.csv',delimiter=',',header=0,index_col=0)

In [3]:
beach_complete.head(10)

,MCurtis-1,Bluffers-10,Rouge-11,Sunnyside-2,Hanlans-3,Gibraltar-4,Centre-5,Wards-6,Cherry-7,Woodbine-8,KewBalmy-9
sampleDate,,,,,,,,,,,
2009-06-01,27.0,12.0,39.0,28.0,11.0,18.0,24.0,63.0,51.0,10.0,10.0
2009-06-02,10.0,14.0,52.0,16.0,10.0,18.0,22.0,18.0,19.0,10.0,10.0
2009-06-03,22.0,11.0,14.0,10.0,10.0,11.0,14.0,10.0,10.0,10.0,10.0
2009-06-04,26.0,10.0,11.0,10.0,10.0,11.0,11.0,10.0,10.0,10.0,10.0
2009-06-05,35.0,10.0,31.0,11.0,10.0,10.0,10.0,12.0,10.0,10.0,10.0
2009-06-06,133.0,13.0,29.0,18.0,12.0,10.0,12.0,16.0,10.0,12.0,10.0
2009-06-07,604.0,16.0,20.0,97.0,37.0,28.0,41.0,47.0,10.0,12.0,11.0
2009-06-08,405.0,17.0,99.0,142.0,31.0,33.0,43.0,39.0,10.0,10.0,12.0
2009-06-09,NaN,37.0,368.0,63.0,11.0,20.0,16.0,11.0,12.0,18.0,34.0


In [4]:
beach_clean = beach_complete.dropna()

In [5]:
beach_clean.apply(pd.to_numeric);

In [6]:
#fig, ax = plt.subplots(6, 2, figsize=(20,40))
#cols = beach_clean.columns
#for i in range(6):
#    for j in range(2):
#        if j == 1 and i == 5:
#            break
#        elif j == 0:
#            ax[i][j].hist(np.log(beach_clean[cols[i]].values), bins=50, histtype='stepfilled')
#            ax[i][j].set(xlabel='E.coli counts',ylabel='Frequency',title=cols[i])
            #set_trace()
#        else:
#            ax[i][j].hist(np.log(beach_clean[cols[i+6]]).values, bins=50, histtype='stepfilled')
#            ax[i][j].set(xlabel='E.coli counts',ylabel='Frequency',title=cols[i+6])

In [7]:
sunnyside = beach_clean[['Sunnyside-2']]

In [8]:
sunnyside.size

668

In [9]:
sunnyside.info()

<class 'pandas.core.frame.DataFrame'>
Index: 668 entries, 2009-06-01 to 2018-09-03
Data columns (total 1 columns):
Sunnyside-2    668 non-null float64
dtypes: float64(1)
memory usage: 10.4+ KB


In [10]:
sunnyside.head()

,Sunnyside-2
sampleDate,
2009-06-01,28.0
2009-06-02,16.0
2009-06-03,10.0
2009-06-04,10.0
2009-06-05,11.0


In [11]:
sunnyside.reset_index(inplace=True)

In [12]:
sunnyside.head()

,sampleDate,Sunnyside-2
0,2009-06-01,28.0
1,2009-06-02,16.0
2,2009-06-03,10.0
3,2009-06-04,10.0
4,2009-06-05,11.0


In [14]:
# plt.hist(sunnyside.values, bins=30, histtype='bar', color='green', label = 'Sunnyside')
# plt.legend()
# plt.show();

In [15]:
# sunnyside.plot(color='red', label = 'Sunnyside')
# plt.legend()
# plt.show();

# Step 2: ACF and PACF plots

In [16]:
# fig, ax = plt.subplots(1, 2, sharey=True, figsize=(15,4))
# plot_acf(sunnyside, lags=50, title='Sunnyside ACF', ax=ax[0])
# plot_pacf(sunnyside, lags=50, title='Sunnyside PACF', ax=ax[1])
# plt.show();

For all 11 beaches, it seems that with a lag=6 there will be very little partial correlation. So we assume lag=6 for both our AR and MA parameters for now.

# Step 5: Setup Train and Test

In [18]:
train_size = int(0.7 * sunnyside['Sunnyside-2'].size)
test_size = int(0.3 * sunnyside['Sunnyside-2'].size)

In [19]:
train_size, test_size

(467, 200)

In [21]:
sunnyside['Label'] = 0
for i in sunnyside.index:
    if i >= train_size:
        sunnyside.loc[i, 'Label'] = 1

/home/rkumar/anaconda3/envs/DS1ENV/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/rkumar/anaconda3/envs/DS1ENV/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Step 6: ARIMA Model

In [ ]:
# hist = list(train.dropna().values)
# pred = []
# residues = []

# for i in range(test.size):
#     model_fit = ARIMA(endog=hist, order=(6,1,1)).fit()
#     pred.append(model_fit.forecast()[0][0])
#     residues.append(model_fit.forecast()[1][0])
#     hist.append(test.values[i])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))
test.plot()
ax.plot(test.index, pred, 'r-')
plt.show();

In [ ]:
rmse = np.sqrt(mean_squared_error(test.dropna().values, pred))
rmse